In [1]:
from pathlib import Path
import numpy as np
import SimpleITK as sitk
%matplotlib notebook
import matplotlib.pyplot as plt
import math
import pandas as pd
import tiffile as tif
import pickle
import os

import multiscale.LINK_system.coordinate as coord
import multiscale.itk.transform as tran
import multiscale.itk.itk_plotting as iplt

# Set up registration
First define the paths to each needed file.  This includes the transform parmaeters, the images themselves, as well as the US position list and the first tile of the microscopy image.

In [2]:
image_dir = Path(r'F:\Research\LINK\Mouse images\2019-05-10')

microscopy_path = Path(image_dir, 'FVB 2mm downsampled-8x.tif')
microscopy_tile_path = Path(image_dir, 'FVB mpm origin.tif')

us_path = Path(image_dir, 'FVB 2mm V11 -7 gauge.tif')
us_pl_path = Path(image_dir, 'FVB 2mm grid.pos')

transform_path = Path(image_dir, 'Coordinate transform.txt')

Next enter in the spacing for each image.  This may be overwritten

In [3]:
us_image = coord.open_us(us_path, us_pl_path, 50, -7)

In [4]:
microscopy_image = coord.open_microscopy(microscopy_path, microscopy_tile_path, downsample_factor=8)

In [5]:
microscopy_image.GetSpacing()

(8.16, 8.16, 25.0)

In [6]:
with open(transform_path, 'rb') as fp:
    coordinate_transform = pickle.load(fp)

In [7]:
us_reg_path = Path(image_dir, 'US Reg to MPM.tif')
mpm_reg_path = Path(image_dir, 'MPM Reg to US.tif')

In [8]:
us_reg = tran.apply_transform_params(microscopy_image, us_image, 
                                     coordinate_transform,
                                     sitk.VersorRigid3DTransform())

In [9]:
mpm_reg = tran.apply_transform_params(us_image, microscopy_image, 
                                     -1*coordinate_transform,
                                     sitk.VersorRigid3DTransform())

In [10]:
sitk.WriteImage(us_reg, str(us_reg_path))

In [11]:
sitk.WriteImage(mpm_reg, str(mpm_reg_path))